In [1]:
import sys
import numpy as np
import pandas as pd
import seaborn as sns

sys.path.append('../..')

from tqdm import tqdm

**Read in the author dimensions from the sample of the dataset**

1k samples from 6 different dataset

In [2]:
dim_df = pd.read_csv('../analyzer/author_dimensions.csv')
dim_df.head()

,1,2,3,4,5,6,author,dataset
0,10.543596,-7.248454,2.280966,-8.765124,-2.670088,-3.314122,AQUU8S55JGY2H,Amazon
1,10.059491,-5.222700,1.584616,-8.513468,-2.373329,-3.363592,52e43081-58d7-8abe-f41c-a2c2e4db7097,News
2,10.185978,-7.119931,2.117917,-9.153395,-1.676906,-3.495074,766dfe84-dae8-e013-b57d-2b4cc532076d,Reddit
3,9.926020,-6.814032,2.528302,-9.115703,-1.872013,-3.415709,6309556.0,Wiki
4,9.184673,-7.017368,1.604767,-9.206371,-1.386046,-3.495074,5641591.0,Wiki


**Shuffle and split 80/20 train/test**

In [3]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(dim_df, test_size=0.2)

print(len(train), len(test))

4800 1200


**Calculate the average dimension for each dataset in the training**

In [4]:
avg_dims = train.groupby('dataset').mean().reset_index()
avg_dims

/var/folders/6y/4xz13f9n0n18x94nx555hjpw0000gr/T/ipykernel_47380/1861058128.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  avg_dims = train.groupby('dataset').mean().reset_index()


,dataset,1,2,3,4,5,6
0,Amazon,11.208473,-6.804783,1.522273,-8.688303,-3.023598,-3.250868
1,BookCorpus,10.806541,-6.160398,1.626548,-8.595062,-2.561371,-3.295711
2,News,10.288745,-6.444490,1.767976,-8.625911,-2.424236,-3.293737
3,Pubmed,9.466732,-6.710433,2.091154,-8.770406,-1.767003,-3.243699
4,Reddit,11.344668,-6.870468,1.528881,-8.603828,-3.146088,-3.338387
5,Wiki,9.829144,-6.853391,1.739694,-9.096389,-2.428304,-3.437767


In [11]:
datasets = avg_dims['dataset'].tolist()

In [5]:
avg_dim_arrs = {}

for index, row in avg_dims.iterrows():
    dims = np.array([row['1'], row['2'], row['3'], 
                     row['4'], row['5'], row['6']])
    avg_dim_arrs[row['dataset']] = dims

In [6]:
def find_closest_dataset(dims_array):
    min_dist, closest_dataset = 1000, ''
    
    for dataset, avg_dim in avg_dim_arrs.items():
        euc_dist = np.linalg.norm(dims_array - avg_dim)
        if euc_dist < min_dist:
            min_dist = euc_dist
            closest_dataset = dataset
            
    return closest_dataset

In [7]:
y_label, y_pred = [], []

In [8]:
for index, row in test.iterrows():
    dims = np.array([row['1'], row['2'], row['3'], 
                     row['4'], row['5'], row['6']])
    
    y_label.append(row['dataset'])
    y_pred.append(find_closest_dataset(dims))

In [9]:
print(len(y_label), len(y_pred))

1200 1200


In [10]:
from sklearn.metrics import classification_report, accuracy_score

print(accuracy_score(y_label, y_pred))

0.5266666666666666
